In [1]:
import pandas as pd

import duckdb

El objetivo de este notebook es crear nuevas clases dataset para distintas DAOs. Vamos a usar el dataset de Andrew en [Kaggle](https://www.kaggle.com/datasets/oiudoiajd/daos-census). Como las tablas son tan grandes (varios GB) no podemos usar `pd.read_parquet`

In [2]:
# Merging all tables into one database
db = duckdb.connect(database=':memory:', read_only=False)
db.execute("CREATE VIEW votes AS SELECT * FROM parquet_scan('{}')".format('data/daos-census/raw/votes.parquet'))
db.execute("CREATE VIEW proposals AS SELECT * FROM parquet_scan('{}')".format('data/daos-census/raw/proposals.parquet'))
db.execute("CREATE VIEW deployments AS SELECT * FROM parquet_scan('{}')".format('data/daos-census/raw/deployments.parquet'))

In [3]:
def get_columns_from_table(table_name):
    return db.execute(f"""
        SELECT column_name, data_type FROM information_schema.columns WHERE table_name='{table_name}'
    """).fetchdf().set_index('column_name')['data_type']

print(get_columns_from_table('votes'))
print(get_columns_from_table('proposals'))

column_name
id                        VARCHAR
proposal_id               VARCHAR
deployment_id             VARCHAR
platform_vote_id          VARCHAR
voter                     VARCHAR
date                    TIMESTAMP
choice                    VARCHAR
weight              DECIMAL(38,4)
Name: data_type, dtype: object
column_name
id                        VARCHAR
deployment_id             VARCHAR
platform_proposal_id      VARCHAR
author                    VARCHAR
date                    TIMESTAMP
votes_count                BIGINT
Name: data_type, dtype: object


In [4]:
db.execute("""
SELECT * FROM deployments
ORDER BY proposals_count DESC
LIMIT 30
""").fetchdf()

,id,platform,platform_deployment_id,name,website,additional,votes_count,proposals_count
0,c37abcdd-a36b-51fc-8fa6-2c3ecb780df0,daohaus,0x1b975a9daf25e7b01e0a6c72d657ff74925327a8,DEAD FoundationsDAO,nan,network: xdai version: 2.0,17738,29681
1,e2ec62ae-9b29-5f3a-a541-58678bb64a1b,snapshot,cakevote.eth,PancakeSwap,https://snapshot.org/#/cakevote.eth,nan,532830,2744
2,41fd8de5-f8e2-5023-86a3-825c49e9ad7f,snapshot,snapshot.dcl.eth,Decentraland,https://snapshot.org/#/snapshot.dcl.eth,decentraland,116560,1962
3,5deda895-4531-539a-b7dc-88797456ded7,daostack,0xe716ec63c5673b3a4732d22909b38d779fa47c3f,xDXdao,nan,nan,5913,1438
4,17dba91a-e68f-50f6-8c1d-2d7fa57ab5fb,snapshot,index-coop.eth,Index Coop,https://snapshot.org/#/index-coop.eth,"indexcoop, https://www.indexcoop.com/",24031,918
5,d3229595-ac4c-55bf-9808-386fc5603bd1,daostack,0x519b70055af55a007110b4ff99b0ea33071c720a,dxDAO,nan,nan,2566,918
6,d8a985e4-3cc2-542a-9cb4-3e024577fb3c,realms,GovER5Lthms3bLBqWub97yVrMmEogzX7xNjdXpPPCVZw_G...,UXDProtocol,NaN,NaN,2562,916
7,7bfe40ed-eb25-546d-b5e0-7db022252f49,daohaus,0x7961b6c69ec7d7203cbecc4f3bf30755713a3822,DAOSquare Grants,nan,network: xdai version: 2.1,702,880
8,51625fff-ef9e-58cd-9dda-0dc68ab75bb2,aragon,0x1c26fde2ce92cd1d932a0afb8367108eac7f369b,NaN,https://etherscan.io/address/0x1c26fde2ce92cd1...,nan,1331,860
9,4ea69b08-fa7f-5daa-96d5-9515ded0076b,snapshot,gm365.eth,gm DAO,https://snapshot.org/#/gm365.eth,nan,91546,730


In [5]:
def best_daos_for_recsys(min_proposals=300, min_voters=50, min_density=0.0001, order_by='proposals_count', limit=10000):
    df = db.execute(f"""
    WITH G AS (
        SELECT deployments.*, COUNT(DISTINCT votes.voter) AS voters_count
        FROM deployments
        LEFT JOIN votes ON (deployments.id = votes.deployment_id)
        WHERE proposals_count > {min_proposals}
        GROUP BY deployments.*
    )
    SELECT *, 
        voters_count + proposals_count AS nodes, 
        2*(votes_count)/((voters_count + proposals_count)*(voters_count + proposals_count-1)) AS density, 
        votes_count/proposals_count AS vpp, 
        votes_count/voters_count AS vpv
    FROM G
    WHERE density > {min_density} AND voters_count > {min_voters}
    ORDER BY {order_by} DESC
    LIMIT {limit}
    """).fetchdf()

    df['milli density'] = 1000 * df['density']

    return df

best_daos_for_recsys().head(10)

,id,platform,platform_deployment_id,name,website,additional,votes_count,proposals_count,voters_count,nodes,density,vpp,vpv,milli density
0,41fd8de5-f8e2-5023-86a3-825c49e9ad7f,snapshot,snapshot.dcl.eth,Decentraland,https://snapshot.org/#/snapshot.dcl.eth,decentraland,116560,1962,7268,9230,0.002737,59.408767,16.037424,2.736675
1,5deda895-4531-539a-b7dc-88797456ded7,daostack,0xe716ec63c5673b3a4732d22909b38d779fa47c3f,xDXdao,nan,nan,5913,1438,92,1530,0.005055,4.111961,64.271739,5.055207
2,d3229595-ac4c-55bf-9808-386fc5603bd1,daostack,0x519b70055af55a007110b4ff99b0ea33071c720a,dxDAO,nan,nan,2566,918,134,1052,0.004642,2.795207,19.149254,4.641605
3,17dba91a-e68f-50f6-8c1d-2d7fa57ab5fb,snapshot,index-coop.eth,Index Coop,https://snapshot.org/#/index-coop.eth,"indexcoop, https://www.indexcoop.com/",24031,918,2871,3789,0.003349,26.177560,8.370254,3.348631
4,4ea69b08-fa7f-5daa-96d5-9515ded0076b,snapshot,gm365.eth,gm DAO,https://snapshot.org/#/gm365.eth,nan,91546,730,7711,8441,0.002570,125.405479,11.872131,2.569999
5,0c40474f-e4a0-540f-ab2f-0451336464f7,snapshot,apollodao.eth,WEALTHDAO,https://snapshot.org/#/apollodao.eth,zhiyuan_2007,4008,631,1041,1672,0.002869,6.351823,3.850144,2.869095
6,7b930871-5e1b-5efe-af9a-dc5413460604,snapshot,huwacoin.eth,HUWA-DAO,https://snapshot.org/#/huwacoin.eth,nan,4151,624,1331,1955,0.002173,6.652244,3.118708,2.173259
7,5ecb2cca-8b8a-5c67-8867-569e5ca0a8ae,snapshot,maturka.eth,9K DAO,https://snapshot.org/#/maturka.eth,https://t.me/Pro_Deng1,102321,592,8170,8762,0.002666,172.839527,12.523990,2.665863
8,899cc74b-f242-5fd2-a270-763436927977,daohaus,0x4570b4faf71e23942b8b9f934b47ccedf7540162,MetaCartel Ventures,nan,network: mainnet version: 2.0,1580,558,77,635,0.007849,2.831541,20.519481,7.849177
9,6276d415-d5f2-5f2f-8d33-3c327f152de6,snapshot,toniya.eth,Klaytn Dex,https://snapshot.org/#/toniya.eth,https://www.klaytn.foundation/,2660,526,869,1395,0.002736,5.057034,3.060990,2.735739


In [6]:
def best_orgs_for_recsys(min_proposals=300, min_voters=50, min_density=0.0005, order_by='proposals_count', limit=10000):
    df = db.execute(f"""
    WITH G AS (WITH Gv AS (
            SELECT *
            FROM deployments
            LEFT JOIN votes ON (deployments.id = votes.deployment_id)
        )
        SELECT 
            name,
            COUNT(DISTINCT Gv.id) AS n_deploys,
            LIST(DISTINCT Gv.id) AS deploys,
            COUNT(DISTINCT Gv.proposal_id) AS proposals_count,
            COUNT(DISTINCT Gv.voter) AS voters_count,
            COUNT(*) AS votes_count,
        FROM Gv
        GROUP BY Gv.name
    )
    SELECT
        *,
        proposals_count + voters_count AS nodes,
        2*(votes_count)/((voters_count + proposals_count)*(voters_count + proposals_count-1)) AS density, 
        votes_count/proposals_count AS vpp, 
        votes_count/voters_count AS vpv
    FROM G
    WHERE density > {min_density} AND proposals_count > {min_proposals} AND voters_count > {min_voters}
    ORDER BY proposals_count DESC
    LIMIT 100000
    """).fetchdf().dropna()

    df['milli density'] = 1000 * df['density']

    return df

best_orgs_for_recsys()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,name,n_deploys,deploys,proposals_count,voters_count,votes_count,nodes,density,vpp,vpv,milli density
0,Decentraland,3,"[7117d5c4-de0e-5e4c-872d-40c3bd4b0200, 36a86d4...",2060,7334,116880,9394,0.002649,56.737864,15.936733,2.649204
1,xDXdao,1,[5deda895-4531-539a-b7dc-88797456ded7],1380,92,5913,1472,0.005462,4.284783,64.271739,5.461569
2,Index Coop,2,"[17dba91a-e68f-50f6-8c1d-2d7fa57ab5fb, a25032f...",874,2871,24032,3745,0.003428,27.496568,8.370603,3.427932
3,dxDAO,1,[d3229595-ac4c-55bf-9808-386fc5603bd1],846,134,2566,980,0.005349,3.033097,19.149254,5.349065
4,Aave,3,"[73c10c98-7499-53b1-8643-c15d8b5dc5b7, b6c95a5...",725,77921,2053071,78646,0.000664,2831.822069,26.348109,0.663875
5,gm DAO,3,"[c3a8aa6c-1652-5834-a355-954bc422f1af, 360a8f2...",710,7712,91548,8422,0.002582,128.940845,11.870851,2.581665
6,9K DAO,1,[5ecb2cca-8b8a-5c67-8867-569e5ca0a8ae],590,8170,102321,8760,0.002667,173.425424,12.523990,2.667081
7,WEALTHDAO,1,[0c40474f-e4a0-540f-ab2f-0451336464f7],585,1041,4008,1626,0.003034,6.851282,3.850144,3.033778
8,MetaCartel Ventures,3,"[899cc74b-f242-5fd2-a270-763436927977, b96c148...",579,124,2132,703,0.008640,3.682211,17.193548,8.640219
9,HUWA-DAO,1,[7b930871-5e1b-5efe-af9a-dc5413460604],572,1331,4151,1903,0.002294,7.256993,3.118708,2.293683


In [7]:
def get_deploys_with_name(name: str):
    return db.execute(f"""
    SELECT *
    FROM deployments
    WHERE name='{name}'
    """).fetchdf()

get_deploys_with_name('Decentraland')

,id,platform,platform_deployment_id,name,website,additional,votes_count,proposals_count
0,41fd8de5-f8e2-5023-86a3-825c49e9ad7f,snapshot,snapshot.dcl.eth,Decentraland,https://snapshot.org/#/snapshot.dcl.eth,decentraland,116560,1962
1,36a86d4c-690b-5da9-87b5-cf2230328b17,aragon,0xf47917b108ca4b820ccea2587546fbb9f7564b56,Decentraland,https://etherscan.io/address/0xf47917b108ca4b8...,nan,319,135
2,7117d5c4-de0e-5e4c-872d-40c3bd4b0200,governor,c584e902-7b8f-4fb0-b742-8fa364c94940,Decentraland,nan,nan,0,0


In [8]:
import torch
from torch_geometric.data import InMemoryDataset, HeteroData

class DAOCensus(InMemoryDataset):
    def __init__(self, root: str, name: str, platform: str=None):
        self._name = name
        self._platform = platform
        
        super().__init__(root)

        self.data = torch.load(self.processed_paths[0])

        from pathlib import Path
        Path(self.processed_paths[0]).unlink()

    def download(self):
        import kaggle
        kaggle.api.dataset_download_cli('oiudoiajd/daos-census', path=self.raw_dir, unzip=True)

    def process(self):
        import pandas as pd
        import duckdb

        db = duckdb.connect(database=':memory:', read_only=False)
        db.execute("CREATE VIEW deployments AS SELECT * FROM parquet_scan('{}')".format(self.raw_paths[0]))
        db.execute("CREATE VIEW votes AS SELECT * FROM parquet_scan('{}')".format(self.raw_paths[1]))
        db.execute("CREATE VIEW proposals AS SELECT * FROM parquet_scan('{}')".format(self.raw_paths[2]))

        cond = f"name='{self._name}'"
        if self._platform:
            cond += f" AND platform='{self._platform}'"

        dfv = db.execute(f"""
        SELECT platform, name, votes.*
        FROM deployments
        LEFT JOIN votes ON (deployments.id = votes.deployment_id)
        WHERE {cond}
        """).fetchdf().rename(columns=lambda x: x.replace('_id', ''))

        dfp = db.execute(f"""
        SELECT platform, name, proposals.*
        FROM deployments
        LEFT JOIN proposals ON (deployments.id = proposals.deployment_id)
        WHERE {cond}
        """).fetchdf().rename(columns=lambda x: x.replace('_id', ''))

        data = HeteroData()
        t = {}

        # display(dfp)
        
        dfv['voter'] = dfv['voter'].str.lower()
        dfp['author'] = dfp['author'].str.lower()

        prop_dtype = pd.api.types.CategoricalDtype(categories=dfp['id'])
        user_dtype = pd.api.types.CategoricalDtype(categories=set(dfv['voter']).union(dfp['author']))

        # voter <-> proposal (dfv)
        dfv['voter'] = dfv['voter'].astype(user_dtype)
        dfv['proposal'] = dfv['proposal'].astype(prop_dtype)

        data['user'].num_nodes = user_dtype.categories.size
        data['user'].voters = dfv['voter'].cat.codes.unique()
        data['proposal'].num_nodes = prop_dtype.categories.size
        
        t = torch.stack([
            torch.LongTensor(dfv['voter'].cat.codes),
            torch.LongTensor(dfv['proposal'].cat.codes)
        ])

        data['user', 'vote', 'proposal'].edge_index = t
        data['proposal', 'vote', 'user'].edge_index = t[(1,0), :]

        # author <-> proposal (dfp)
        dfp['author'] = dfp['author'].astype(user_dtype)
        dfp['id'] = dfp['id'].astype(prop_dtype)
        t = torch.stack([
            torch.LongTensor(dfp['author'].cat.codes),
            torch.LongTensor(dfp['id'].cat.codes),
        ])

        data['user'].authors = dfp['author'].cat.codes.unique()
        data['user', 'creates', 'proposal'].edge_index = t
        data['proposal', 'creates', 'user'].edge_index = t[(1,0), :]

        data.validate()
        assert not data.is_directed()
        assert not data.has_isolated_nodes()

        db.close()
        torch.save(data, self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ["deployments.parquet", "votes.parquet", "proposals.parquet"]

    @property
    def processed_file_names(self) -> str:
        return f"daostack_votes_{self._name}.pt"

data = DAOCensus("./data/daos-census", 'Decentraland', 'snapshot')
data[0]

Processing...
Done!


HeteroData(
  user={
    num_nodes=7270,
    voters=[7268],
    authors=[2],
  },
  proposal={ num_nodes=1962 },
  (user, vote, proposal)={ edge_index=[2, 116560] },
  (proposal, vote, user)={ edge_index=[2, 116560] },
  (user, creates, proposal)={ edge_index=[2, 1962] },
  (proposal, creates, user)={ edge_index=[2, 1962] }
)